# Leonardo's Self-Propelled Cart: The First Automobile

```{admonition} Historical Context
:class: note
Leonardo's self-propelled cart, designed around 1495, represents the first programmable automobile in history. Found in the Codex Atlanticus, this spring-powered vehicle could move autonomously and even be "programmed" to follow predetermined paths.
```

Explore how Leonardo's mechanical design work anticipated programmable vehicles centuries before the modern automobile.

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Cart simulation class
class CartSimulation:
    @staticmethod
    def simulate(spring_energy=500, cart_mass=50, wheel_radius=0.3, friction_coeff=0.1):
        """Simulate cart motion with energy conservation."""
        g = 9.81
        
        # Rolling resistance calculation
        rolling_resistance = friction_coeff * cart_mass * g
        
        # Maximum theoretical distance (no air resistance)
        max_distance = spring_energy / rolling_resistance
        
        # Maximum velocity (at start)
        max_velocity = np.sqrt(2 * spring_energy / cart_mass)
        
        # Time to stop (approximation)
        time_to_stop = max_distance / (max_velocity / 2)
        
        # Create motion profile
        time = np.linspace(0, time_to_stop, 100)
        
        # Velocity profile (linear decay approximation)
        velocity = max_velocity * (1 - time / time_to_stop)
        velocity = np.maximum(velocity, 0)  # Can't go negative
        
        # Position profile
        position = np.cumsum(velocity) * (time[1] - time[0]) if len(time) > 1 else [0]
        
        return {
            'max_distance': max_distance,
            'max_velocity': max_velocity,
            'time_to_stop': time_to_stop,
            'time_profile': time,
            'velocity_profile': velocity,
            'position_profile': position,
            'energy_efficiency': max_distance / (spring_energy / rolling_resistance)
        }

In [ ]:
# Interactive cart simulator
def create_cart_simulator():
    # Parameter controls
    spring_energy_slider = widgets.FloatSlider(
        value=500, min=100, max=1500, step=50,
        description='Spring Energy (J):',
        style={'description_width': 'initial'}
    )
    
    cart_mass_slider = widgets.FloatSlider(
        value=50, min=20, max=100, step=5,
        description='Cart Mass (kg):',
        style={'description_width': 'initial'}
    )
    
    wheel_radius_slider = widgets.FloatSlider(
        value=0.3, min=0.1, max=0.6, step=0.05,
        description='Wheel Radius (m):',
        style={'description_width': 'initial'}
    )
    
    friction_slider = widgets.FloatSlider(
        value=0.1, min=0.05, max=0.3, step=0.01,
        description='Friction Coefficient:',
        style={'description_width': 'initial'}
    )
    
    output = widgets.Output()
    
    def update_simulation(change=None):
        with output:
            clear_output(wait=True)
            
            # Run simulation
            results = CartSimulation.simulate(
                spring_energy=spring_energy_slider.value,
                cart_mass=cart_mass_slider.value,
                wheel_radius=wheel_radius_slider.value,
                friction_coeff=friction_slider.value
            )
            
            # Display results
            print(f"🏎️ Self-Propelled Cart Performance:")
            print(f"Maximum Distance: {results['max_distance']:.1f} m")
            print(f"Maximum Velocity: {results['max_velocity']:.1f} m/s ({results['max_velocity']*3.6:.1f} km/h)")
            print(f"Operating Time: {results['time_to_stop']:.1f} seconds")
            print(f"Energy Efficiency: {results['energy_efficiency']:.1%}")
            
            # Create motion visualization
            fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 10))
            
            # Velocity vs time
            ax1.plot(results['time_profile'], results['velocity_profile'], 'b-', linewidth=2)
            ax1.set_xlabel('Time (s)')
            ax1.set_ylabel('Velocity (m/s)')
            ax1.set_title('Cart Velocity Over Time')
            ax1.grid(True)
            
            # Position vs time
            ax2.plot(results['time_profile'], results['position_profile'], 'r-', linewidth=2)
            ax2.set_xlabel('Time (s)')
            ax2.set_ylabel('Position (m)')
            ax2.set_title('Cart Position Over Time')
            ax2.grid(True)
            
            # Energy analysis
            kinetic_energy = 0.5 * cart_mass_slider.value * results['velocity_profile']**2
            potential_energy = spring_energy_slider.value - kinetic_energy
            
            ax3.plot(results['time_profile'], kinetic_energy, 'g-', label='Kinetic Energy', linewidth=2)
            ax3.plot(results['time_profile'], potential_energy, 'orange', label='Remaining Spring Energy', linewidth=2)
            ax3.set_xlabel('Time (s)')
            ax3.set_ylabel('Energy (J)')
            ax3.set_title('Energy Distribution Over Time')
            ax3.legend()
            ax3.grid(True)
            
            plt.tight_layout()
            plt.show()
            
            # Design comparison
            create_design_comparison(results)
    
    # Connect sliders to update function
    spring_energy_slider.observe(update_simulation, 'value')
    cart_mass_slider.observe(update_simulation, 'value')
    wheel_radius_slider.observe(update_simulation, 'value')
    friction_slider.observe(update_simulation, 'value')
    
    display(widgets.VBox([
        widgets.HTML('<h3>🏎️ Cart Simulation Parameters</h3>'),
        spring_energy_slider,
        cart_mass_slider,
        wheel_radius_slider,
        friction_slider
    ]))
    display(output)
    
    # Initial simulation
    update_simulation()

def create_design_comparison(base_results):
    """Compare different design variations."""
    
    # Compare different configurations
    configs = {
        'Leonardo\'s Design': {'energy': 500, 'mass': 50, 'friction': 0.1},
        'Lightweight Version': {'energy': 500, 'mass': 30, 'friction': 0.1},
        'High Energy': {'energy': 1000, 'mass': 50, 'friction': 0.1},
        'Low Friction': {'energy': 500, 'mass': 50, 'friction': 0.05}
    }
    
    config_names = list(configs.keys())
    distances = []
    velocities = []
    
    for config in configs.values():
        result = CartSimulation.simulate(
            spring_energy=config['energy'],
            cart_mass=config['mass'],
            friction_coeff=config['friction']
        )
        distances.append(result['max_distance'])
        velocities.append(result['max_velocity'])
    
    # Create comparison chart
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Distance comparison
    bars1 = ax1.bar(config_names, distances, color=['brown', 'blue', 'green', 'orange'])
    ax1.set_ylabel('Maximum Distance (m)')
    ax1.set_title('Range Comparison')
    ax1.tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for bar, distance in zip(bars1, distances):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + 10,
                f'{distance:.0f}m', ha='center', va='bottom')
    
    # Velocity comparison
    bars2 = ax2.bar(config_names, velocities, color=['brown', 'blue', 'green', 'orange'])
    ax2.set_ylabel('Maximum Velocity (m/s)')
    ax2.set_title('Speed Comparison')
    ax2.tick_params(axis='x', rotation=45)
    
    # Add value labels on bars
    for bar, velocity in zip(bars2, velocities):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height + 0.2,
                f'{velocity:.1f}m/s', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()

create_cart_simulator()

## Historical Innovation and Modern Relevance

### Leonardo's Design Features

```{admonition} Key Design Concepts
:class: tip
Leonardo's cart introduced several automotive concepts centuries early:
- **Energy Storage**: Coiled springs as power source
- **Programmable Path**: Adjustable cam mechanism for steering
- **Differential Gear**: Allows wheels to turn at different speeds
- **Brake System**: Hand-operated stopping mechanism
```

### Technical Analysis

#### Energy Conservation Principle
$$E_{spring} = \frac{1}{2}kx^2 = \frac{1}{2}mv^2 + F_{friction} \cdot d$$

Where:
- $E_{spring}$: Stored spring energy
- $k$: Spring constant
- $x$: Spring compression
- $m$: Cart mass
- $v$: Velocity
- $F_{friction}$: Friction force
- $d$: Distance traveled

### Modern Applications

Leonardo's principles influence contemporary technology:
- **Autonomous Vehicles**: Self-guidance systems
- **Energy Recovery**: Spring energy storage in cars
- **Robotics**: Programmable motion control
- **Toy Industry**: Wind-up and spring-powered vehicles

In [ ]:
# Programming demonstration
def demonstrate_programmable_path():
    """Show how Leonardo's cart could be programmed."""
    
    # Create a simple path programming interface
    path_html = """
    <div style="background: #f0f8ff; border: 2px solid #4169e1; border-radius: 10px; padding: 20px; margin: 20px 0;">
        <h3 style="color: #0000cd;">🎛️ Programmable Path System</h3>
        <p><strong>Leonardo's Innovation:</strong> The cart used a system of wooden cams and pegs to control steering.</p>
        
        <h4>Programming Method:</h4>
        <ol>
            <li><strong>Cam Wheel:</strong> Wooden disk with adjustable pegs</li>
            <li><strong>Steering Linkage:</strong> Pegs engage with steering mechanism</li>
            <li><strong>Path Control:</strong> Peg positions determine turn angles</li>
            <li><strong>Timing:</strong> Cam rotation speed controls turn timing</li>
        </ol>
        
        <p><em>This represents the first programmable vehicle in history - predating modern computers by 500 years!</em></p>
    </div>
    """
    
    display(HTML(path_html))
    
    # Simple path visualization
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Example programmed path
    x_path = [0, 5, 5, 10, 10, 15, 15, 20]
    y_path = [0, 0, 5, 5, 0, 0, -5, -5]
    
    ax.plot(x_path, y_path, 'b-', linewidth=3, marker='o', markersize=8, label='Programmed Path')
    ax.set_xlabel('Distance (m)')
    ax.set_ylabel('Lateral Position (m)')
    ax.set_title('Example Programmable Cart Path')
    ax.grid(True, alpha=0.3)
    ax.legend()
    ax.set_aspect('equal')
    
    # Add annotations for turns
    turn_points = [(5, 0), (10, 5), (15, 0)]
    for i, (x, y) in enumerate(turn_points):
        ax.annotate(f'Turn {i+1}', xy=(x, y), xytext=(x+1, y+1),
                   arrowprops=dict(arrowstyle='->', color='red'),
                   fontsize=10, color='red')
    
    plt.tight_layout()
    plt.show()

demonstrate_programmable_path()

In [ ]:
# Historical context and summary
summary_html = """
<div style="background: #fff8dc; border: 2px solid #d4af37; border-radius: 10px; padding: 20px; margin: 20px 0;">
    <h3 style="color: #8b4513;">📚 Historical Impact & Learning Summary</h3>
    
    <h4>Leonardo's Achievement:</h4>
    <ul>
        <li><strong>First Automobile:</strong> Self-propelled land vehicle (1495)</li>
        <li><strong>First Robot:</strong> Autonomous, programmable motion</li>
        <li><strong>Advanced Mechanics:</strong> Differential gears, steering systems</li>
        <li><strong>Energy Storage:</strong> Efficient spring power transmission</li>
    </ul>
    
    <h4>Modern Validation:</h4>
    <p>In 2004, Paolo Galluzzi's team at the Institute and Museum of the History of Science in Florence 
    built a working replica based on Leonardo's designs. The cart successfully demonstrated:</p>
    <ul>
        <li>Autonomous forward motion for approximately 40 meters</li>
        <li>Programmable steering through cam-controlled mechanism</li>
        <li>Reliable braking system</li>
    </ul>
    
    <h4>Educational Value:</h4>
    <ul>
        <li><strong>Physics:</strong> Energy conservation, mechanics, friction</li>
        <li><strong>Engineering:</strong> Mechanical design, automation, control systems</li>
        <li><strong>History:</strong> Renaissance innovation, technological progress</li>
        <li><strong>Computer Science:</strong> Early programming concepts</li>
    </ul>
    
    <p style="font-style: italic; text-align: center; margin-bottom: 0;">
        "This cart is a machine that moves by itself with various motions." - Leonardo da Vinci
    </p>
</div>
"""

display(HTML(summary_html))